In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
direct = ""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        direct = os.path.join(dirname, filename)
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Gender: 0 - male; 1 - female

Ethnicity: 0 - white; 1 - afro-american; 2 - asian; 3 - indian (brown); 4 - unkown for me :D

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from PIL import Image
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from IPython.display import Image
from skimage.transform import resize
from skimage.io import imread, imshow

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

In [ ]:
df = pd.read_csv(direct)

In [ ]:
df.head()


In [ ]:
df.shape

## EDA for the data

This part is necessary for every project. You should know your data and the way of dependency of its features.  

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (18, 8))
fig.suptitle("Count for each label")

sns.countplot(ax = axes[0], x = df.ethnicity)
axes[0].set_title("Ethnicity")

sns.countplot(ax = axes[1], x = df.gender)
axes[1].set_title("Gender")

plt.show()

In [ ]:
plt.figure(figsize=(18,8))
sns.histplot(df.age, kde=True)
plt.show()

In [ ]:
plt.figure(figsize=(18,8))
sns.kdeplot(x=df.ethnicity)
plt.show()

In [ ]:
df.corr()

We saw for the Ethnicity we have kinda unbalanced data but for gender prediction will not be a problem. 
Both of those problems are classification ones but for age prediction we have rregression.
First of all I saw really old people here but are they really that old or dataset mistake?
Should i take them for training ?

In [ ]:
df.age.describe()

In [ ]:
def pixel_reshape(column):
    column = np.array([x for x in column.str.split()], dtype ="float32")
    column = np.reshape(column, (-1,48,48,1))
    return np.array(column)

In [ ]:
old_people = df[df.age > 95]

In [ ]:
oldImgs = pixel_reshape(old_people.pixels)

In [ ]:
oldImgs.shape

In [ ]:
for i in range(1, 15):
    plt.imshow(oldImgs[i], cmap = "gray")
    plt.show()  

So for now we can see that they are actually old people but we have very few of them. We can se duplicated as well. Ether out regression model is going to be good on predicting the age are we are going to need data augmentation for those years.... 

#### Scale Data

### Age classification
train, val, test split the data based on Gender

In [ ]:
TRAIN_PCT = 0.8
VAL_PCT = 0.1
TEST_PCT = 0.1


In [ ]:
def get_stratify_split_by(df, category):
    train_data = pd.DataFrame()
    val_data = pd.DataFrame()
    test_data = pd.DataFrame()
    
    for (label, group_img) in df.groupby(category):
        group_img = group_img.sample(len(group_img), random_state=24)

        train_data_end_index = int(len(group_img) * TRAIN_PCT)
        val_data_end_index = train_data_end_index + int(len(group_img) * VAL_PCT)


        train_data_in_group = group_img[:train_data_end_index]
        val_data_in_group = group_img[train_data_end_index:val_data_end_index]
        test_data_in_group = group_img[val_data_end_index :]

        print(len(train_data_in_group), len(val_data_in_group), len(test_data_in_group))

        train_data = train_data.append(train_data_in_group)
        val_data = val_data.append(val_data_in_group)
        test_data = test_data.append(test_data_in_group)
    return train_data, val_data, test_data

In [ ]:
train_data, val_data, test_data = get_stratify_split_by(df, "gender")

In [ ]:
train_data.ethnicity.value_counts(), val_data.ethnicity.value_counts(), test_data.ethnicity.value_counts()

In [ ]:
train_data.shape, val_data.shape, test_data.shape

In [ ]:
X_train = (pixel_reshape(train_data.pixels)) / 255
X_val = (pixel_reshape(val_data.pixels)) / 255
X_test = (pixel_reshape(test_data.pixels)) / 255

In [ ]:
model_one = Sequential([
    Conv2D(16, padding="same", kernel_size=(2, 2), input_shape=((48,48,1))),
    MaxPool2D((2, 2)),
    Conv2D(32, kernel_size=(3, 3), padding="same"),
    MaxPool2D((2, 2)),
    Conv2D(64, kernel_size=(3, 3), padding="valid"),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(1, activation="sigmoid")
])

In [ ]:
model_one.summary()

In [ ]:
model_one.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model_one_hist = model_one.fit(X_train, train_data.gender, batch_size=64, epochs=8, validation_data=(X_val, val_data.gender))

In [ ]:
plt.plot(model_one_hist.history['loss'])
plt.plot(model_one_hist.history['val_loss'])

In [ ]:
plt.plot(model_one_hist.history['accuracy'])
plt.plot(model_one_hist.history['val_accuracy'])

In [ ]:
model_one.evaluate(X_test, test_data.gender)

In [ ]:
preds = [np.round(i, 0) for i in model_one.predict(X_test)]

In [ ]:
sns.heatmap(confusion_matrix(test_data.gender, preds), annot=True, fmt="d")

In [ ]:
for i in range(6):
    pic = X_test[i] * 255
    plt.title(f"True label: {test_data.gender.iloc[i]}, Predicted label: {preds[i]}")
    plt.imshow(pic)
    plt.show()

### Ethnicity

In [ ]:
model_two = Model(inputs=model_one.input, outputs=model_one.layers[-2].output)
model_two = Sequential([
    model_two,
    Dense(len(set(train_data.ethnicity)), activation =  "softmax")
])

In [ ]:
model_two.summary()

In [ ]:
model_two.compile(optimizer=Adam(), loss = "sparse_categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
model_two_hist = model_two.fit(X_train, train_data.ethnicity,
                               batch_size=64, epochs=16,
                               validation_data=(X_val, val_data.ethnicity),
                               callbacks=[EarlyStopping(patience=2, restore_best_weights=True, monitor="val_accuracy")])

In [ ]:
plt.plot(model_two_hist.history['loss'])
plt.plot(model_two_hist.history['val_loss'])

In [ ]:
plt.plot(model_two_hist.history['accuracy'])
plt.plot(model_two_hist.history['val_accuracy'])

In [ ]:
model_two.evaluate(X_test, test_data.ethnicity)

In [ ]:
preds = [np.argmax(i) for i in model_two.predict(X_test)]

In [ ]:
sns.heatmap(confusion_matrix(test_data.ethnicity, preds), annot=True, fmt="d")

For now we saw that the most common class is the most reconised by the model and we have 0 correct prediction of the 4th one. 

### AGE PREDICTION

In [ ]:
age_prediciton_model = Sequential([
    Conv2D(16, padding="same", kernel_size=(2, 2), input_shape=((48,48,1))),
    MaxPool2D((2, 2)),
    Conv2D(32, kernel_size=(3, 3), padding="same"),
    Conv2D(32, kernel_size=(3, 3), padding="same"),
    MaxPool2D((2, 2)),
    Conv2D(64, kernel_size=(3, 3), padding="valid"),
    Conv2D(64, kernel_size=(3, 3), padding="valid"),
    BatchNormalization(),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="linear")
])

In [ ]:
age_prediciton_model.summary()

In [ ]:
age_prediciton_model.compile(optimizer=Adam(), loss="mean_absolute_error", metrics=['mean_absolute_error'])

In [ ]:
age_prediciton_model.fit(X_train, train_data.age,
                               batch_size=120, epochs=12,
                               validation_data=(X_val, val_data.age))

In [ ]:
age_prediciton_model.evaluate(X_test, test_data.age)

In [ ]:
preds = age_prediciton_model.predict(X_test)

for i in range(20):
    print(f"True age: {test_data.age.iloc[i]} vs Predicted_age: {int(preds[i])}")

In the end we saw how to create classification and regression models.
Gender model accuracy is nearly 89% and the age prediction with 7 MAE.

The thing is that those pictures were zoomed and most of them missed the hair factor. If we had pics with hair maby our models would be better at predicting (or worse if whe have High bias for women with long hair).

The ethnicity didn't do as well as I would like, but maby some Image augmentation would do a little bit better.
I will continue to edit this notebook with more text... (to be continue.. :) )